# Genetic Algorithm

In this notebook we learn an abstraction using genetic algorithms.

Code is cloned and adapted from https://github.com/chengxi600/RLStuff/blob/master/Genetic%20Algorithms/8Queens_GA.ipynb

## Importing libraries

We import the basic libraries.

In [1]:
import numpy as np
import itertools
from scipy.spatial import distance
from scipy.sparse import coo_matrix

from pgmpy.models import BayesianNetwork as BN
from pgmpy.factors.discrete import TabularCPD as cpd

## Defining constants

We first define reproducibility and output constants

In [2]:
np.random.seed(1985)

np.set_printoptions(precision=4,suppress=True)

We then specify GA constants.

In [3]:
POPULATION_SIZE = 10
CROSSOVER_POINTS = 1
PARENTS_PER_OFFSPRING = 2
MUTATION_RATE = 0.1

NON_SURJ_PENALTY = 100

## Problem statement

We want to learn an abstraction $(R,a,\alpha_X)$ between two structural causal models $\mathcal{M}$ and $\mathcal{M'}$. We are given $R,a$, and we want to learn the collection of maps $\alpha_X$, which have the form of binary matrices.

Our base model is:

$$
\begin{array}{ccccc}
S & \overset{\mu_T}{\longrightarrow} & T & \overset{\mu_C}{\longrightarrow} & C
\end{array}
$$

while the abstracted model is:
$$
\begin{array}{ccc}
S' & \overset{\mu_{T'}}{\longrightarrow} & T' & \overset{\mu_{C'}}{\longrightarrow} & C'
\end{array}
$$

where $\mu_\cdot$ are the mechanism of the structural causal model, expressed again as matrices.

The set of relevant variables $R$ includes all the base variables ($S,T,C$) and $a$ trivially maps every base variable to the corresponding abstracted variable ($S\mapsto S', T\mapsto T', C\mapsto C'$).

Our aim is to learn the abstractions $\alpha_{S'},\alpha_{T'},\alpha_{C'}$ such that the following three diagrams commute:

(i) $$
\begin{array}{ccc}
S& \overset{\mu_T}{\rightarrow} & T \\
\alpha_{S'}{\downarrow}& &{\downarrow}\alpha_{T'} \\
S'& \overset{\mu_{T'}}{\rightarrow} & T'
\end{array}
$$
meaning that we want the following equation to hold: $\alpha_{T'} \cdot \mu_T = \mu_{T'} \cdot \alpha_{S'}$;

(ii)
$$
\begin{array}{ccc}
T& \overset{\mu_C}{\rightarrow} & C \\
\alpha_{T'}{\downarrow}& &{\downarrow}\alpha_{C'} \\
T'& \overset{\mu_{C'}}{\rightarrow} & C'
\end{array}
$$
meaning that we want the following equation to hold: $\alpha_{C'} \cdot \mu_C = \mu_{C'} \cdot \alpha_{T'}$;

(iii)
$$
\begin{array}{ccc}
S& \overset{\mu_C\cdot\mu_T}{\rightarrow} & C \\
\alpha_{S'}{\downarrow}& &{\downarrow}\alpha_{C'} \\
S'& \overset{\mu_{C'} \cdot \mu_{T'}}{\rightarrow} & C'
\end{array}
$$
meaning that we want the following equation to hold: $\alpha_{C'} \cdot (\mu_C\cdot\mu_T) = (\mu_{T'} \cdot \mu_{C'}) \cdot \alpha_{S'}$.

## Model definition

We start defining the matrices for the mechanisms $\mu_T, \mu_C, \mu_{T'}, \mu_{C'}$.

In [4]:
mech_low_T = np.array([[.6,.55,.1,.1],[.3,.25,.4,.4],[.1,.2,.5,.5]], dtype=np.float32)
mech_low_C = np.array([[.7,.7,.4],[.3,.3,.6]], dtype=np.float32)
mech_high_T = np.array([[.9,.8,.5],[.1,.2,.5]], dtype=np.float32)
mech_high_C = np.array([[.7,.4],[.3,.6]], dtype=np.float32)

Then we implement the base model $\mathcal{M}$:

In [5]:
M0 = BN([('S','T'),('T','C')])

cpdS = cpd(variable='S',
          variable_card=4,
          values=[[.25],[.25],[.25],[.25]],
          evidence=None,
          evidence_card=None)
cpdT = cpd(variable='T',
          variable_card=3,
          values=mech_low_T,
          evidence=['S'],
          evidence_card=[4])
cpdC = cpd(variable='C',
          variable_card=2,
          values=mech_low_C,
          evidence=['T'],
          evidence_card=[3])

M0.add_cpds(cpdS,cpdT,cpdC)
M0.check_model()

True

And model $\mathcal{M'}$:

In [6]:
M1 = BN([('S_','T_'),('T_','C_')])

cpdS = cpd(variable='S_',
          variable_card=3,
          values=[[.25],[.5],[.25]],
          evidence=None,
          evidence_card=None)
cpdT = cpd(variable='T_',
          variable_card=2,
          values=mech_high_T,
          evidence=['S_'],
          evidence_card=[3])
cpdC = cpd(variable='C_',
          variable_card=2,
          values=mech_high_C,
          evidence=['T_'],
          evidence_card=[2])

M1.add_cpds(cpdS,cpdT,cpdC)
M1.check_model()

True

Notice that, for the sake of this problem, the complete definition of the two models is not required (and we will not be using the variables $M0$ and $M1$ later). Yet, this gives a full picture of the setting, and it allows to see how the abstracted models is a smaller version (fewer outcomes) of the base model.

For reference, we also formally define the unique solution of our learning problem. Again, this data is provided only for completeness, and it will not be used in learning.

In [7]:
R = ['S','T','C']
a = {'S': 'S_',
     'T': 'T_',
     'C': 'C_'}
alphas = {"S_": np.array([[1,0,0,0],
                [0,1,0,0],
                [0,0,1,1]]),
         "T_": np.array([[1,1,0],
                [0,0,1]]),
         "C_": np.array([[1,0],
                [0,1]])}

from src.legacy.SCMMappings_1_0 import Abstraction
A = Abstraction(M0,M1,R,a,alphas)

A complete description of the model and the abstraction may be found in previous notebook.

## Learning setup

The aim of our problem is to learn three abstraction maps ($\alpha_{S'},\alpha_{T'},\alpha_{C'}$). Each one can be encoded as a matrix with following properties:
- *large rectangular*: the number of rows is less or equal to the number of columns;
- *binary*: values are in $\{0,1\}$;
- *stochastic*: columns sum up to $1$;
- *surjective*: every row contains at least a $1$.

We then define, as constant for our problem, the matrices we want to learn and their dimensions.

In [8]:
MATRICES = [[3,4],[2,3],[2,2]]
N_MATRICES = len(MATRICES)

## Encoding

We will consider two representations for our matrices $\alpha_{S'},\alpha_{T'},\alpha_{C'}$:
- **Binary matrix:** $\alpha_{\cdot}$ is represented as a binary matrix; this representation is useful to quickly assess the properties described above;
- **Discrete vector:** in analogy with the *n-queens problem*, we also compress a matrix into a discrete vector; an $(m \times n)$ matrix is translated into a $n$-values vector where each entry takes a value in $[0,m-1]$ representing the position of the zero.

The two representations are equivalent; we have the following duality:
$$
\left[\begin{array}{cccc}
0 & 1 & 1 & 0\\
1 & 0 & 0 & 0\\
0 & 0 & 0 & 1
\end{array}\right]\leftrightarrows\left[\begin{array}{cccc}
1 & 0 & 0 & 2\end{array}\right]
$$

We implement two function to convert between these representations:

In [9]:
def convert_population_vector2matrix(population):
    
    new_population = []
    for ind in population:
        
        new_individual = []
        for i in range(N_MATRICES):
            rows = ind[i]
            cols = np.arange(MATRICES[i][1])
            data = np.ones(MATRICES[i][1])
            new_matrix = coo_matrix((data, (rows,cols)), shape=(MATRICES[i][0],MATRICES[i][1])).toarray()
        
            new_individual.append(new_matrix)
        
        new_population.append(new_individual)
    
    return new_population

def convert_population_matrix2vector(population):
    
    new_population = []
    for ind in population:
        
        new_individual = []
        for i in range(N_MATRICES):
            matrix = coo_matrix(ind[i])
            rows = matrix.row
            cols = matrix.col
            idxs = np.argsort(cols)
            new_vector = matrix.row[idxs]
            
            new_individual.append(new_vector)
        
        new_population.append(new_individual)
    
    return new_population

**TODO:** testing the conversion functions

In [10]:
def test_conversion(population):
    test_population = convert_population_matrix2vector(convert_population_vector2matrix(population))
    
    return False

## Specifying individuals and populations

In our implementation, we will define:
- *individual* as a single solution for $\alpha_{S'},\alpha_{T'},\alpha_{C'}$;
- *population* as a collection of such individuals

We define a function to generate a random starting population.

In [11]:
def generate_population():
    new_population = []

    for _ in range(POPULATION_SIZE):
        new_individual = []
        for i in range(N_MATRICES):
            new_vector = np.random.randint(MATRICES[i][0],size=MATRICES[i][1])
            new_individual.append(new_vector)
        new_population.append(new_individual)
    
    return new_population

We create a population and take a look at it.

In [12]:
pop = generate_population()
pop

[[array([1, 1, 2, 0]), array([1, 0, 0]), array([0, 0])],
 [array([2, 2, 2, 1]), array([1, 1, 0]), array([1, 0])],
 [array([1, 0, 2, 1]), array([0, 0, 0]), array([0, 0])],
 [array([2, 0, 2, 2]), array([1, 1, 1]), array([1, 1])],
 [array([1, 1, 0, 1]), array([0, 1, 0]), array([1, 1])],
 [array([2, 0, 2, 0]), array([0, 1, 1]), array([0, 1])],
 [array([1, 0, 1, 0]), array([0, 0, 1]), array([0, 1])],
 [array([0, 0, 0, 0]), array([1, 0, 0]), array([0, 0])],
 [array([0, 1, 1, 0]), array([0, 0, 0]), array([0, 1])],
 [array([1, 2, 2, 0]), array([0, 0, 1]), array([0, 0])]]

## Fitness function

We evaluate a candidate solution according to two criteria.

(i) First, we want the solution to satisfy the properties listed above for the $\alpha_\cdot$ matrices. Notice that the first three properties (large rectangular, binary, stochastic) are trivially guaranteed by our encoding; surjectivity, however has to be checked. We then introduce a function that check for surjectivity and returns how many matrices $\alpha_\cdot$ do not satisfy this property.

In [13]:
def check_surjectivity(individual):
    penalty = 0
    for i in range(N_MATRICES): 
        if not np.all(np.sum(individual[i],axis=1)>=1): penalty+=1
    
    return penalty

(ii) Second, we want the learned abstraction matrices to guarantee the following identities:

$$\alpha_{T'} \cdot \mu_T = \mu_{T'} \cdot \alpha_{S'}$$
$$\alpha_{C'} \cdot \mu_C = \mu_{C'} \cdot \alpha_{T'}$$
$$\alpha_{C'} \cdot (\mu_C\cdot\mu_T) = (\mu_{T'} \cdot \mu_{C'}) \cdot \alpha_{S'}$$

when this does not happen, we estimate the quality/fitness of the solution as the Jensen-Shannon distance between lhs and rhs:
$$D_{JSD}(\alpha_{T'} \cdot \mu_T, \mu_{T'} \cdot \alpha_{S'}) +$$
$$D_{JSD}(\alpha_{C'} \cdot \mu_C, \mu_{C'} \cdot \alpha_{T'}) +$$
$$D_{JSD}(\alpha_{C'} \cdot (\mu_C\cdot\mu_T), (\mu_{T'} \cdot \mu_{C'}) \cdot \alpha_{S'})$$

We implement it into a function.

In [14]:
def compute_jsd(individual):
    jsd = 0
    jsd += np.max(distance.jensenshannon(np.dot(individual[1],mech_low_T),np.dot(mech_high_T,individual[0])))
    jsd += np.max(distance.jensenshannon(np.dot(individual[2],mech_low_C),np.dot(mech_high_C,individual[1])))
    jsd += np.max(distance.jensenshannon(np.dot(individual[2],np.dot(mech_low_C,mech_low_T)),np.dot(np.dot(mech_high_C,mech_high_T),individual[0])))
    return jsd

The total fitness is the sum of the number of violation of surjectivity (scaled by a large number to penalize this violation) plus the JS distance.

In [15]:
def fitness_score(individual):
    score = - NON_SURJ_PENALTY*check_surjectivity(individual) - compute_jsd(individual)
    return score

We use a helper printing function to compute the fitness over a population.

In [16]:
def print_fitnesses(population):
    pop_matrix = convert_population_vector2matrix(population)
    for ind in pop_matrix:
        print(fitness_score(ind))

In [17]:
print_fitnesses(pop)

-101.27762383771446
-100.55059013283281
-201.24180851681268
-301.69601475947366
-201.40258316569674
-100.90874367293848
-100.40609848296018
-201.27065315437133
-200.5757060434273
-101.27762383208756


## Genetic operators

Next we define genetic operators, most of which are just inherited from https://github.com/chengxi600/RLStuff/blob/master/Genetic%20Algorithms/8Queens_GA.ipynb.

### Selection operator

We define a simple function to select parents proportionally to their fitness

In [18]:
def selection(population):
    parents = []
    
    pop_matrix = convert_population_vector2matrix(population)
    for ind in pop_matrix:
        if np.random.randint(N_MATRICES*NON_SURJ_PENALTY) > -fitness_score(ind):
            parents.append(ind)
    
    return convert_population_matrix2vector(parents)

We check the operator on our initial population.

In [19]:
sel = selection(pop)
print_fitnesses(sel)

-200.5757060434273
-101.27762383208756


### Crossover operator

We define a crossover operator by allowing two parents to swap matrices. In other words, a cross-over point $p$ is selected in $[0,N_MATRICES-1]$. The new child will inherit matrices $0$ to $p$ from the first parent, and $p$ to $N_MATRICES-1$ from the second parent. 

Notice that this cross-over operator does not affect the internal structure of individual matrices.

In [20]:
def crossover(parents,verbose=False):
    
    offsprings = []
    permutations = list(itertools.permutations(parents, PARENTS_PER_OFFSPRING))
    for p in permutations:
        if verbose: print(p)
    
    for perm in permutations:
        if verbose: print('\nCandidates in the perm: \n {0} \n {1}'.format(perm[0],perm[1]))
        
        offspring0 = perm[0]
        offspring1 = perm[1]
        offspring = []
        crosspoint = np.random.randint(N_MATRICES)
        if verbose: print('Crosspont: {0}'.format(crosspoint))
        
        for i in range(0,crosspoint):
            offspring.append(offspring0[i])
        for i in range(crosspoint,len(offspring1)):
            offspring.append(offspring1[i])
        
        if verbose: print('Crossed-over offspring: {0}'.format(offspring))
        offsprings.append(offspring)
        if verbose: print(offsprings)
        
    return offsprings

We run an instance of cross-over in verbose mode on a sub-population to examine its behaviour.

In [21]:
sub_pop = pop[0:3]
crossover(sub_pop,verbose=True)

([array([1, 1, 2, 0]), array([1, 0, 0]), array([0, 0])], [array([2, 2, 2, 1]), array([1, 1, 0]), array([1, 0])])
([array([1, 1, 2, 0]), array([1, 0, 0]), array([0, 0])], [array([1, 0, 2, 1]), array([0, 0, 0]), array([0, 0])])
([array([2, 2, 2, 1]), array([1, 1, 0]), array([1, 0])], [array([1, 1, 2, 0]), array([1, 0, 0]), array([0, 0])])
([array([2, 2, 2, 1]), array([1, 1, 0]), array([1, 0])], [array([1, 0, 2, 1]), array([0, 0, 0]), array([0, 0])])
([array([1, 0, 2, 1]), array([0, 0, 0]), array([0, 0])], [array([1, 1, 2, 0]), array([1, 0, 0]), array([0, 0])])
([array([1, 0, 2, 1]), array([0, 0, 0]), array([0, 0])], [array([2, 2, 2, 1]), array([1, 1, 0]), array([1, 0])])

Candidates in the perm: 
 [array([1, 1, 2, 0]), array([1, 0, 0]), array([0, 0])] 
 [array([2, 2, 2, 1]), array([1, 1, 0]), array([1, 0])]
Crosspont: 1
Crossed-over offspring: [array([1, 1, 2, 0]), array([1, 1, 0]), array([1, 0])]
[[array([1, 1, 2, 0]), array([1, 1, 0]), array([1, 0])]]

Candidates in the perm: 
 [array(

[[array([1, 1, 2, 0]), array([1, 1, 0]), array([1, 0])],
 [array([1, 1, 2, 0]), array([1, 0, 0]), array([0, 0])],
 [array([2, 2, 2, 1]), array([1, 0, 0]), array([0, 0])],
 [array([2, 2, 2, 1]), array([1, 1, 0]), array([0, 0])],
 [array([1, 0, 2, 1]), array([0, 0, 0]), array([0, 0])],
 [array([1, 0, 2, 1]), array([0, 0, 0]), array([1, 0])]]

### Mutation operator

Finally, we define a mutation operator than simply flip a value with a given probability. Since the mutation operator is the only operator changing the internal structure of a matrix, we may have to set its probability to a relatively high value.

In [22]:
def mutate(population,verbose=False):
    for ind in population:
        for i in range(N_MATRICES):
            for j in range(len(ind[i])):
                if np.random.rand() < MUTATION_RATE:
                    if verbose: print('Mutation in {0},{1},{2}!'.format(ind,i,j))
                    ind[i][j] = np.random.randint(MATRICES[i][0])
    
    return population

**TODO:** make it efficient taking out the loops.

## Genetic algorithm

We now bring all our functions togehter, and define a single step in the genetic algorithm. It starts from a given population, selects parents, performs crossover, performs mutation, subselects survivors according to fitness, returns a new generation.

In [23]:
def evolution_step(population):
    parents = selection(population)

    offsprings = crossover(parents)

    offsprings = mutate(offsprings)

    new_gen = offsprings
    for ind in population:
        new_gen.append(ind)
    idxs = np.argsort(list(map(fitness_score,convert_population_vector2matrix(new_gen))))[::-1][0:POPULATION_SIZE]
    new_gen = [new_gen[i] for i in idxs]

    return new_gen

## Simulation

We are now ready to setup the simulation.

We define a termination condition that let us run until we find the solution.

In [24]:
def print_found_goal(population, to_print=True):
    for ind in convert_population_vector2matrix(population):
        score = fitness_score(ind)
        if to_print:
            print('Score: {0}'.format(score))
            #print(f'{ind}. Score: {score}')
        if np.isclose(score,0):
            if to_print:
                print('Solution found')
            return True
    
    if to_print:
        print('Solution not found')
    return False

And, finally, we run the simulation by generating a starting population and taking one GA step at a time.

In [25]:
generation = 0

population = generate_population()
    
while not print_found_goal(population):
    print(f'Generation: {generation}')
    print_found_goal(population)
    population = evolution_step(population)
    generation += 1

Score: -0.5536314351129212
Score: -100.28997681794073
Score: -101.03276236101
Score: -201.5031063807406
Score: -100.90874367563687
Score: -0.6634064438239131
Score: -200.99418570294102
Score: -1.0928541482958514
Score: -201.25626034322113
Score: -301.4246577740111
Solution not found
Generation: 0
Score: -0.5536314351129212
Score: -100.28997681794073
Score: -101.03276236101
Score: -201.5031063807406
Score: -100.90874367563687
Score: -0.6634064438239131
Score: -200.99418570294102
Score: -1.0928541482958514
Score: -201.25626034322113
Score: -301.4246577740111
Solution not found
Score: -0.4060984917401661
Score: -0.4060984917401661
Score: -0.5536314351129212
Score: -0.6634064438239131
Score: -0.7125686466322962
Score: -0.8201327495941557
Score: -1.0928541482958514
Score: -100.28997681794073
Score: -100.90874367563687
Score: -101.03276236101
Solution not found
Generation: 1
Score: -0.4060984917401661
Score: -0.4060984917401661
Score: -0.5536314351129212
Score: -0.6634064438239131
Score: -0.

The algorithm converged to the exact solution in $12$ generations (this, of course, has little statistical value at the moment...).

Let see the final population.

In [26]:
population

[[array([0, 1, 2, 2], dtype=int32),
  array([0, 0, 1], dtype=int32),
  array([0, 1], dtype=int32)],
 [array([1, 0, 2, 2], dtype=int32),
  array([0, 0, 1], dtype=int32),
  array([0, 1], dtype=int32)],
 [array([1, 0, 2, 2], dtype=int32),
  array([0, 0, 1], dtype=int32),
  array([0, 1], dtype=int32)],
 [array([1, 0, 2, 2], dtype=int32),
  array([0, 0, 1], dtype=int32),
  array([0, 1], dtype=int32)],
 [array([1, 0, 2, 2], dtype=int32),
  array([0, 0, 1], dtype=int32),
  array([0, 1], dtype=int32)],
 [array([1, 0, 2, 2], dtype=int32),
  array([0, 0, 1], dtype=int32),
  array([0, 1], dtype=int32)],
 [array([1, 0, 2, 2], dtype=int32),
  array([0, 0, 1], dtype=int32),
  array([0, 1], dtype=int32)],
 [array([0, 0, 1, 2], dtype=int32),
  array([0, 0, 1], dtype=int32),
  array([0, 1], dtype=int32)],
 [array([0, 1, 1, 2], dtype=int32),
  array([0, 0, 1], dtype=int32),
  array([0, 1], dtype=int32)],
 [array([1, 0, 1, 2], dtype=int32),
  array([0, 0, 1], dtype=int32),
  array([0, 1], dtype=int32)]]

The solution is the first individual:

In [27]:
 population[0]

[array([0, 1, 2, 2], dtype=int32),
 array([0, 0, 1], dtype=int32),
 array([0, 1], dtype=int32)]

Or, in a more familiar formatting, in matrix format:

In [28]:
convert_population_vector2matrix(population)[0]

[array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 1.]]),
 array([[1., 1., 0.],
        [0., 0., 1.]]),
 array([[1., 0.],
        [0., 1.]])]

which indeed correspond to the known solution we have written down above:

In [29]:
alphas

{'S_': array([[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 1]]),
 'T_': array([[1, 1, 0],
        [0, 0, 1]]),
 'C_': array([[1, 0],
        [0, 1]])}